# Importing Data

In [1]:
import pandas as pd

In [25]:
housing_data = pd.read_csv('housing.csv')

In [26]:
housing_data.head()

,housingMedianAge,totalRooms,totalBedrooms,population,households,medianIncome,medianHouseValue
0,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0
1,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0
2,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0
3,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0
4,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0


# Splitting The Dataset 

In [4]:
labels = housing_data['medianHouseValue']

In [5]:
features = housing_data.drop('medianHouseValue',axis=1)

In [6]:
from sklearn.model_selection import train_test_split

In [7]:
X_train,X_test,y_train,y_test = train_test_split(features,labels,test_size=0.3,random_state=0)

# Feature Scaling the data

In [8]:
from sklearn.preprocessing import MinMaxScaler

In [9]:
scaler = MinMaxScaler()

In [10]:
scaler.fit(X_train)

MinMaxScaler(copy=True, feature_range=(0, 1))

In [11]:
X_train = pd.DataFrame(data=scaler.transform(X_train),columns=X_train.columns,index=X_train.index)

In [12]:
X_test = pd.DataFrame(data=scaler.transform(X_test),columns=X_test.columns,index=X_test.index)

# Creating Feature Columns

In [13]:
housing_data.columns

Index(['housingMedianAge', 'totalRooms', 'totalBedrooms', 'population',
       'households', 'medianIncome', 'medianHouseValue'],
      dtype='object')

In [14]:
import tensorflow as tf

In [15]:
age = tf.feature_column.numeric_column('housingMedianAge')
rooms = tf.feature_column.numeric_column('totalRooms')
bedrooms = tf.feature_column.numeric_column('totalBedrooms')
population = tf.feature_column.numeric_column('population')
households = tf.feature_column.numeric_column('households')
income = tf.feature_column.numeric_column('medianIncome')

In [16]:
feat_cols = [age,rooms,bedrooms,population,households,income]

# Creating Input Function

In [17]:
input_func = tf.estimator.inputs.pandas_input_fn(x=X_train,y=y_train,batch_size=10,num_epochs=1000,shuffle=True)

# Creating The Estimator model

In [21]:
model = tf.estimator.DNNRegressor(hidden_units=[5,5,5],feature_columns=feat_cols)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_tf_random_seed': 1, '_save_checkpoints_secs': 600, '_keep_checkpoint_max': 5, '_session_config': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_model_dir': '/tmp/tmpxdhr2owk', '_log_step_count_steps': 100, '_keep_checkpoint_every_n_hours': 10000}


# Training the model

In [19]:
model.train(input_fn=input_func,steps=10000)

INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into /tmp/tmpspwa1_0r/model.ckpt.
INFO:tensorflow:step = 1, loss = 4.52781e+11
INFO:tensorflow:global_step/sec: 410.559
INFO:tensorflow:step = 101, loss = 3.74581e+11 (0.250 sec)
INFO:tensorflow:global_step/sec: 351.731
INFO:tensorflow:step = 201, loss = 2.81582e+11 (0.289 sec)
INFO:tensorflow:global_step/sec: 349.319
INFO:tensorflow:step = 301, loss = 5.60148e+11 (0.282 sec)
INFO:tensorflow:global_step/sec: 369.687
INFO:tensorflow:step = 401, loss = 7.35767e+11 (0.273 sec)
INFO:tensorflow:global_step/sec: 322.261
INFO:tensorflow:step = 501, loss = 5.65773e+11 (0.312 sec)
INFO:tensorflow:global_step/sec: 328.693
INFO:tensorflow:step = 601, loss = 3.24329e+11 (0.301 sec)
INFO:tensorflow:global_step/sec: 343.102
INFO:tensorflow:step = 701, loss = 3.57458e+11 (0.294 sec)
INFO:tensorflow:global_step/sec: 335.558
INFO:tensorflow:step = 801, loss = 1.49529e+11 (0.296 sec)
INFO:tensorflow:global_step/sec: 386

INFO:tensorflow:global_step/sec: 299.651
INFO:tensorflow:step = 8101, loss = 1.62063e+11 (0.328 sec)
INFO:tensorflow:global_step/sec: 331.58
INFO:tensorflow:step = 8201, loss = 5.74476e+10 (0.303 sec)
INFO:tensorflow:global_step/sec: 329.625
INFO:tensorflow:step = 8301, loss = 1.30008e+11 (0.299 sec)
INFO:tensorflow:global_step/sec: 340.805
INFO:tensorflow:step = 8401, loss = 5.20432e+10 (0.303 sec)
INFO:tensorflow:global_step/sec: 342.045
INFO:tensorflow:step = 8501, loss = 6.96101e+10 (0.290 sec)
INFO:tensorflow:global_step/sec: 349.053
INFO:tensorflow:step = 8601, loss = 8.33227e+10 (0.289 sec)
INFO:tensorflow:global_step/sec: 347.102
INFO:tensorflow:step = 8701, loss = 1.4029e+11 (0.284 sec)
INFO:tensorflow:global_step/sec: 424.467
INFO:tensorflow:step = 8801, loss = 1.08287e+11 (0.235 sec)
INFO:tensorflow:global_step/sec: 412.611
INFO:tensorflow:step = 8901, loss = 1.79792e+11 (0.241 sec)
INFO:tensorflow:global_step/sec: 412.487
INFO:tensorflow:step = 9001, loss = 1.62323e+11 (0.2

# Create Predict input function

In [49]:
predict_input_func = tf.estimator.inputs.pandas_input_fn(x=X_test,batch_size=10,num_epochs=1,shuffle=False)

In [50]:
pred = model.predict(input_fn=predict_input_func)

In [51]:
predictions = list(pred)

INFO:tensorflow:Restoring parameters from /tmp/tmpb2bsj2lo/model.ckpt-10000


# Calculate Root Mean Squared Errors - RMSE

In [52]:
final_pred = []
for pred in predictions:
    final_pred.append(pred['predictions'])

In [53]:
from sklearn.metrics import mean_squared_error

In [54]:
mean_squared_error(y_test,final_pred)**0.5

104157.12193101335